In [ ]:
from pulp import *
import pandas as pd
from models import *
import numpy as np
from results_analysis import *

In [ ]:
test_mode = False

## Load in data

In [ ]:
def load_res_profile(file:str)->pd.Series:

    res_norm = pd.read_csv(f'data/renewable_profiles/{file}',skiprows=3,index_col=0)
    res_norm.index = pd.to_datetime(res_norm.index)
    res_norm = res_norm['electricity']
    return res_norm


from typing import Dict

def generate_res_input(capacities:Dict[str,int],source_files:Dict[str,str]):
    data = {}
    for name, source in source_files.items():
        data[name] = load_res_profile(source)
    
    profiles_df = pd.concat(data.values(),keys=data.keys(),axis=1)
    return Renewables(
        names=list(data.keys()),
        capacities=capacities,
        profiles=profiles_df
        )

source_files = {
    'Wind':'wind_amsterdam.csv',
    'Solar':'solar_amsterdam.csv',
}

res = generate_res_input(
    capacities={'Solar':100,'Wind':300},
    source_files=source_files
)

if test_mode:
    res.generation = res.generation.iloc[:2]

In [ ]:
time_index = range(res.generation.shape[0])
time_index

## Define load and renewables size (MW)

In [ ]:
load = Load(
    capacity=80,
    profile=pd.Series(np.ones(len(time_index)),index=res.generation.index)
)
load.load

In [ ]:
storage = Storage(
    efficiency=0.7,
    minimum_duration=10,
    capex_power=800,
    capex_capacity=100,
)

In [ ]:
inputs = ScenarioInputs(
    name='BaseCase',
    start_date=res.generation.index[0],
    time_index=time_index,
    renewables =res,
    load=load,
    storage=storage
)

In [ ]:
inputs.renewables.generation

In [ ]:
scenario = ScenarioManager(inputs=inputs)

In [ ]:
scenario.lpmodel

In [ ]:
scenario.lpmodel.variables()

In [ ]:
scenario.lpmodel.solve()

In [ ]:
solve = scenario.solve()

## Results

In [ ]:
scenario.lpmodel.status

In [ ]:


get_design_results(scenario)



In [ ]:


get_operational_batteryflow_results(scenario)



In [ ]:


operational_results = get_full_operation_data(scenario)
operational_results

In [ ]:
fig = px.line(operational_results,title=f'a')

In [ ]:

plot_results(scenario,figure_kwargs={'height':900})